In [ ]:
import pandas as pd
from difflib import SequenceMatcher
import math

filepath = 'BI.xlsx'
columns_to_inspect = ["NOM NETTOYE", "nom_client",
                      "adresse (si présente dans factures)", "responsable", "telephone", "email"]
columns_to_sort = ["CODE CE", "code_postal"]
group_by = "CODE CE"  # those with same values for column are grouped
sheet_name = "CLIENTS (BI WeDev)"
header = 0  # row with column titles



In [ ]:
datas = pd.read_excel(filepath, sheet_name=sheet_name,
                      header=header)
    

In [ ]:
columns = [col for col in datas]
datas = datas.sort_values(columns_to_sort)
unique_values = datas[group_by].unique() #pour faire des groupes pas code ce

In [130]:
all_ids_clients_to_check = []
for unique_value in unique_values:
    group = datas.loc[datas[group_by] == unique_value]
    client_ids_tocheck = similar_client_ids(group)
    all_ids_clients_to_check += client_ids_tocheck
    print(len(all_ids_clients_to_check))



0
62
96
96
96
100
124
124
180
303
591
721
776
844
891
947
1371
1371
1681
1690
1748
2066
2068
2068
2126
2126
2211
2215
2223
2233
2443
2468
2468
2474
2574
2603
2631
2709
2740
2740
2740
2798
2808
2817
2824
2826
2826
2830
2830
2838
2838
2838
2864
2870
2874
3165
3165
3216
3216
3218
3218
3220
3220
3224
3310
3312
3467
3502
3512
3520
3526
3563
3790
3817
3819
3833
3847
4119
4149
4149
4209
4215
4266


In [ ]:
print(columns)
datas.info()

In [131]:
def str_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def list_simililarity(l1,l2):
    return math.prod([str_similarity(str(elt1), str(elt2)) for (elt1, elt2) in zip(l1,l2)])

def similar_client_ids(df):

    similar_client_ids = []

    n_indexes = len(df.index)
    for i in range(n_indexes - 1):
        row_i = df.iloc[i]
        for j in range(i+1, n_indexes):
            row_j = df.iloc[j]

            inspect_row_i = row_i[columns_to_inspect]
            inspect_row_j = row_j[columns_to_inspect]
            similarity_ij = list_simililarity(inspect_row_i,inspect_row_j)

            if similarity_ij > 0.5:
                row_i_client_id = row_i["client_id"]
                row_j_client_id = row_j["client_id"]
                # print(row_i_client_id, row_j_client_id, end = "|")

                if (row_i_client_id in similar_client_ids):
                    similar_client_ids.insert(similar_client_ids.index(row_i_client_id) + 1, row_j_client_id)
                else:
                    similar_client_ids.append(row_i_client_id)
                    similar_client_ids.append(row_j_client_id)
                break
    return similar_client_ids

def df_from_client_ids(df,client_ids):
    new_df = df[df["client_id"].isin(client_ids)]
    new_df.to_csv("BI_duplicates.xlsx")
    return new_df

def df_without_client_ids(df,client_ids):
    return df[~df["client_id"].isin(client_ids)]
    

In [ ]:
duplicates = df_from_client_ids(group,client_ids_tocheck)
df_without_client_ids(group,client_ids_tocheck)